# Antiphon, Arb, and Archimedes #

Antiphon, possibly together with Bryson of Heraclea, proposed an argument for computing $\pi$ (or, possibly, defining it) by means of the _areas_ of inscribed and circumscribed polygons.  They seem to have also introduced the idea of doubling the number of sides of the $n$-gons to improve their estimates.  [Archimedes of Syracuse](https://en.wikipedia.org/wiki/Archimedes) famously took the argument concretely and actually computed the areas of polygons of $96 = 3\cdot 2^5$ sides, rounding his answers _down_ for the inscribed figure and rounding his answers _up_ for the circumscribed figure.  This is the first documented use of what we now know as _interval arithmetic_.  It also seems to us to be the first use of _backward error_: we want to compute the area or perimeter of a circle, and instead we compute the exact area or perimeter of a nearby curve! 

This is actually a hard historical argument to be confident in.  It seems likely that the Egyptians and Babylonians were very well-versed in approximate computation, so perhaps they had earlier used what we would now call perturbation arguments.  However, we are not aware of any examples of such.


As an aside, at this point in time, it was well appreciated that if the area of a unit circle was $\pi$, then the perimeter was $\tau = 2\pi$.  Hence the formulas for perimeter and area were supposed to be conversible. But the idea that the _area_ of the inscribed $n$-gon is less than the _area_ of the circle is obvious: one has cut pieces of the circle off to make the inscribed polygon. Similarly, the _area_ of the circumscribed polygon is obviously larger than the area of the circle, because you can cover the circle completely with it.  The arguments for length are somewhat more subject to doubt, and in particular it's not immediately clear that the perimeter of the inscribed polygon is less than the perimeter of the circumscribed polygon. So, we think about area here.

Let $a_n$ be the _area_ of a regular $n$-gon _inscribed_ in the unit circle.  Similarly, let $A_n$ be the area of a regular $n$-gon _circumscribed_ about the unit circle.

We derived the following formulae, which you might find entertaining to rederive for yourselves.
\begin{align}
A_{2n} &= \frac{2A_n}{1+\sqrt{1+\left(\frac{A_n}{n}\right)^2}} \\
a_{2n} &= \frac{A_n}{\sqrt{1+\left(\frac{A_n}{n}\right)^2}}
\end{align}

We have that the inscribed and circumscribed triangles give
\begin{align}
A_3 &= 3\sqrt{3}\\
a_3 &= \frac{3\sqrt{3}}{4} .
\end{align}

Copyright (c) 2024 Robert M. Corless

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the “Software”), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [1]:
A := table():
a := table():
A[3] := 3*sqrt(3):
a[3] := 3*sqrt(3)/4:
for k to 5 do
  n := 3*2^(k-1);
  A[2*n] := 2*A[n]/(1+sqrt(1+(A[n]/n)^2));
  a[2*n] := A[n]/sqrt(1+(A[n]/n)^2);
end do:

kilobytes used=1872, alloc=5424, time=0.41

In [2]:
Digits := 15:
bounds := [a[2*n],A[2*n]];

| | |
|:-:|:-:|
|48*3^(1/2)/(1+2/3*3^(1/2))/(1+1/3*(9+3/(1+2/3*3^(1/2))^2)^(1/2))/(1+1/3*(9+3/(1+2/3*3^(1/2))^2/(1+1/3*(9+3/(1+2/3*3^(1/2))^2)^(1/2))^2)^(1/2))/(9+3/(1+2/3*3^(1/2))^2/(1+1/3*(9+3/(1+2/3*3^(1/2))^2)^(1/2))^2/(1+1/3*(9+3/(1+2/3*3^(1/2))^2/(1+1/3*(9+3/(1+2/3*3^(1/2))^2)^(1/2))^2)^(1/2))^2)^(1/2)|32*3^(1/2)/(1+2/3*3^(1/2))/(1+1/3*(9+3/(1+2/3*3^(1/2))^2)^(1/2))/(1+1/3*(9+3/(1+2/3*3^(1/2))^2/(1+1/3*(9+3/(1+2/3*3^(1/2))^2)^(1/2))^2)^(1/2))/(1+1/3*(9+3/(1+2/3*3^(1/2))^2/(1+1/3*(9+3/(1+2/3*3^(1/2))^2)^(1/2))^2/(1+1/3*(9+3/(1+2/3*3^(1/2))^2/(1+1/3*(9+3/(1+2/3*3^(1/2))^2)^(1/2))^2)^(1/2))^2)^(1/2))|


In [3]:
bf := evalf[5](bounds);

| | |
|:-:|:-:|
|3.1393|3.1428|


Those numbers do not take account of rounding errors.  Rounding errors are actually negligible here because the number of sides is so low.  Nonetheless if one wants a computer-aided proof that the true area is between the bounds, then we have to do as Archimedes did and round down for the lower limit and up for the upper limit. 

There is now a way to do this in native Maple, based on the [wonderful program Arb by Fredrik Johansson](https://fredrikj.net/arb/).  This beats the old [interval arithmetic package by Amanda Connell and myself](https://interval.louisiana.edu/reliable-computing-journal/1993/interval-computations-1993-2-pp-120-134.pdf), by more than a country mile.  In particular, that 1993 work reports a computer taking more than an hour to reproduce the computation that we do below in a flash.

In Maple, you just make each atomic element something called a "RealBox" and then most of Maple's operators (In Maple 2024, not sqrt, unfortunately, so we have to use $x^{1/2}$; this may get fixed in later versions) just work and automatically track the rounding errors.

In [4]:
A := table():
a := table():
Sqrt := x -> x^(1/2); # sqrt does things Arb doesn't like
A[3] := RealBox( 3*sqrt(3) ):
a[3] := RealBox( 3*sqrt(3)/4 ):
for k to 5 do
  n := 3*2^(k-1);
  A[2*n] := 2*A[n]/(1+Sqrt(1+(A[n]/n)^2));
  a[2*n] := A[n]/Sqrt(1+(A[n]/n)^2);
end do:

                               Sqrt := x -> x

In [5]:
bounds := [a[2*n],A[2*n]];

| | |
|:-:|:-:|
|RealBox(3.13935020304687,8.18589518416821e-014)|RealBox(3.14271459964536,7.76126804917689e-014)|


In [6]:
Center(a[2*n])-Radius(a[2*n]), Center(A[2*n])+Radius(A[2*n])

3.13935020304679, 3.14271459964544

Like Archimedes, we have rounded down for the lower limit, and rounded up for the upper.  Now we can let this iteration run for a lot longer, and use polygons with millions of sides.

By experiment, we find that at 15 Digits we can't use more than 23 doublings, giving us more than $800$ million sides; more than that and the rounding errors dominate.  If we chose to use higher settings of Digits, we could be more accurate.

In [7]:
A := table():
a := table():
Sqrt := x -> x^(1/2); # sqrt does things Arb doesn't like
A[3] := RealBox( 3*sqrt(3) ):
a[3] := RealBox( 3*sqrt(3)/4 ):
for k to 23 do
  n := 3*2^(k-1);
  A[2*n] := 2*A[n]/(1+Sqrt(1+(A[n]/n)^2));
  a[2*n] := A[n]/Sqrt(1+(A[n]/n)^2);
end do:

                               Sqrt := x -> x

In [8]:
(lower,upper) := ( Center(a[2*n])-Radius(a[2*n]), Center(A[2*n])+Radius(A[2*n]) );

3.14159265358946, 3.14159265359013

In [9]:
upper-lower;

6.71906974503145e-013

In [10]:
2*n;

25165824

[Ludolph van Ceulen](https://en.wikipedia.org/wiki/Ludolph_van_Ceulen) (1540-1610) computed $\pi$ essentially this way to $35$ decimal digits; a prodigious feat of computation, albeit somewhat pointlessly because there were already better methods available, though not widely known at the time.  Still, it's fun to retrace his footsteps.

In [11]:
Digits := 100;
A := table():
a := table():
Sqrt := x -> x^(1/2): # sqrt does things Arb doesn't like
A[3] := RealBox( 3*sqrt(3) ):
a[3] := RealBox( 3*sqrt(3)/4 ):
for k to 60 do
  n := 3*2^(k-1);
  A[2*n] := 2*A[n]/(1+Sqrt(1+(A[n]/n)^2));
  a[2*n] := A[n]/Sqrt(1+(A[n]/n)^2);
end do:
(lower,upper) := ( Center(a[2*n])-Radius(a[2*n]), Center(A[2*n])+Radius(A[2*n]) ):
upper-lower;

                                 Digits := 100

.2591836663304908011030848367521194487990489084926997811939180386e-35

In [12]:
2*n;

3458764513820540928

Putting $59$ in the loop doesn't quite give us $35$ digits.  The difference between the upper and lower limits would then be $0.1036 \cdot 10^{-34}$ which just misses (unless we take the average of the upper and lower value).  But $60$ gives less than $0.3\cdot 10^{-35}$, which means we have $35$ digits guaranteed to be correct.  The number of sides is absurd, though: $3.46\cdot 10^{18}$, so each polygonal side would be smaller than a proton's width!

If $\theta_n = 2\pi/n$ then $a_n = \frac{n}{2}\sin\theta_n$ and $A_n = n\tan(\theta_n/2)$.

In [13]:
n := 'n':
asympt( n*sin(2*Pi/n)/2, n );

Pi-2/3*Pi^3/n^2+2/15*Pi^5/n^4+O(1/n^6)

In [14]:
asympt( n*tan(Pi/n), n );

kilobytes used=5605, alloc=10801, time=0.87

Pi+1/3*Pi^3/n^2+2/15*Pi^5/n^4+O(1/n^6)

From this analysis we see that $(a_n+2A_n)/3 = \pi + O(1/n^4)$ eliminates the first term of the error (and leaves a slight overestimate of $\pi$). So by being more clever (for instance, using Richardson extrapolation) we could have found as accurate value of $\pi$ as Ludolph did but using many fewer sides.



In [15]:
hey := (a[3*2^60]+2*A[3*2^60])/3;

In [16]:
Center(hey);

3.14159265358979323846264338327950288419716939937510582097494459230781640657131353126375688673780387352022

In [17]:
Radius(hey);

.5296936e-97

In [18]:
Center(hey)-evalf(Pi);

.285104532635722061395686806e-72

In [20]:
3 - (265/153)^2;

2/23409

In [22]:
(1351/780)^2 - 3;

1/608400